In [2]:
import pandas as pd
x=pd.read_csv("C:/Users/charumathi/Downloads/Paper2/Sentiment.csv")
df=pd.DataFrame(x)
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [4]:
df.shape

(13871, 21)

In [16]:
df=df[["text","sentiment"]]

In [17]:
df.isnull().sum()

text         0
sentiment    0
dtype: int64

In [18]:
df.duplicated().sum()

2891

In [22]:
df=df.drop_duplicates()

### Q1. Print the total number of positive and negative sentiments.

In [25]:
print("Total Positive Sentiments",len(df[df['sentiment']=='Positive']))
print("Total Negative Sentiments",len(df[df['sentiment']=='Negative']))

Total Positive Sentiments 1709
Total Negative Sentiments 6129


### Q2. Build a sequential LSTM model to predict positive and negative sentiments.

In [32]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\charumathi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
lower=text
print(text.lower())

he is a great leader.


In [35]:
text.split()

['He', 'is', 'a', 'great', 'leader.']

In [36]:
from nltk.tokenize import word_tokenize
token=word_tokenize(text)
print(token)

['He', 'is', 'a', 'great', 'leader', '.']


In [37]:
from nltk.probability import FreqDist
fdist=FreqDist(token)
fdist

FreqDist({'He': 1, 'is': 1, 'a': 1, 'great': 1, 'leader': 1, '.': 1})

In [39]:
import textblob
from textblob import TextBlob
TextBlob(text).ngrams(2)

[WordList(['He', 'is']),
 WordList(['is', 'a']),
 WordList(['a', 'great']),
 WordList(['great', 'leader'])]

In [40]:
from nltk import word_tokenize
from nltk.corpus import stopwords

In [41]:
nltk.download('stopwords')
a=set(stopwords.words('english'))
x=word_tokenize(text.lower())
print(x)

['he', 'is', 'a', 'great', 'leader', '.']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\charumathi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [50]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\charumathi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['text'] = df.text.apply(lemmatize_text)
df

,text,sentiment
1,RT @ScottWalker: Didn't catch full #GOPdebate ...,Positive
3,RT @RobGeorge: That Carly Fiorina trending -- ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On first day I r...",Positive
6,RT @warriorwoman91: I liked happy I heard goin...,Negative
...,...,...
13845,"RT @the818: Let's play ""how fast distance Trum...",Negative
13847,RT @mjtbaum: GOD making appearance #GOPDebate?...,Positive
13857,"This I watch Fox News, they're asshole #GopDeb...",Negative
13859,"Best line #GOPDebate ""Immigration without assi...",Positive


In [47]:
s = 0.0
for i in df['text']:
    word_list = i.split()
    s = s + len(word_list)
print("Average length of each text : ",s/df.shape[0])
pos = 0
for i in range(df.shape[0]):
    if df.iloc[i]['sentiment'] == 'positive':
        pos = pos + 1
neg = df.shape[0]-pos
print("Percentage of text with positive sentiment is "+str(pos/df.shape[0]*100)+"%")
print("Percentage of text with negative sentiment is "+str(neg/df.shape[0]*100)+"%")

Average length of each text :  12.240877774942588
Percentage of text with positive sentiment is 0.0%
Percentage of text with negative sentiment is 100.0%


### Q3.Based on the model, check the sentiment for the following two sentences


### a. 'He is a great leader.'b. 'He is a terrible leader.'

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)from keras .preprocessing.text import Tokenizer
sentence = ['He is a great leader','He is a terrible leader.' ]
# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)
# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)
# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print(sentence[i])
    if pred_labels[i] == 1:
        s = 'Positive'
    else:
        s = 'Negative'
    print("Predicted sentiment : ",s)
